In [1]:
# parameters cell
input_dir_pdbqt = None
output_dir_sdf = None
convert_files = False
pdb_file = ''
ligands_file = ''
ligands_files_dir = None
reference_file = ''
write_scores = True
score_col = 'PLIF_SCORE'

In [2]:
# Parameters
input_dir_pdbqt = "/data/docking/GDP/docked"
output_dir_sdf = "/data/docking/GDP/docked/sdf"
pdb_file = "/data/docking/GDP/5O33_nolig_prepared.pdb"
ligands_file = ""
ligands_files_dir = "/data/docking/GDP/docked/sdf"
reference_file = "/data/GDP_frags/GDP.sdf"
write_scores = True
convert_files = True


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import sys, os, glob
sys.path.append('/code/tools')
from scoring.plif import PlifScore
from misc import vina_pdbqt_to_sdf, is_interactive
from ui.file_widgets import DockButtons, PathSelector, InputBoxSet
import scrapbook as sb

# Scoring vina results: PLIF (protein-ligand interaction fingerprint) against reference

---

### 1. Convert pdbqt files to sdf files 

---

input directory = the place where all of the pdbqt files from a docking run are  
output directory = where to write the sdfs out to 

In [5]:
if is_interactive():
    iconv = InputBoxSet(tlabel='input dir:')
    display(iconv.render())


In [6]:
if is_interactive():
    oconv = InputBoxSet(tlabel='output dir:')
    display(oconv.render())

In [7]:
if is_interactive():
    input_dir_pdbqt = iconv.inbox.value
    output_dir_sdf = oconv.inbox.value

if convert_files:
    infiles = glob.glob(os.path.join(input_dir_pdbqt, '*.pdbqt'))

    if not os.path.isdir(output_dir_sdf):
        os.makedirs(output_dir_sdf)

        [vina_pdbqt_to_sdf(ifile=i, 
                           output=os.path.join(output_dir_sdf, i.split('/')[-1].replace('.pdbqt', '.sdf'))) 
         for i in infiles]

---
### 2. Select some input files for the PLIF scoring
---

#### 2.1.  Select the protein and reference ligand: 
*(N.B. run the next 2 cells to show the file dialogues)*
1. click on the arrow in the dropdown box at the top
2. select the relevant file in the dialogue 
3. click the button next to either protein, query ligands or reference to set the file

---

#### 2.2.  Select the query molecules to score against the reference ligand:

**There are two options here:** 

1. Use one file to specify which ligands to score: use the buttons as described above
2. Specify a directory of sdf files of multiple files of ligands to score: type the directory to search in the 'multiple dir' input box, and click 'set'


In [8]:
if is_interactive():
    p = PathSelector(start_dir=os.getcwd())
    display(p.accord)

    b = DockButtons(path_selector=p)
    display(b.render_buttons())

In [9]:
if is_interactive():
    pdb_file = b.pdb_button.description
    ligands_file = b.lig_button.description
    ref_file = b.ref_button.description

In [10]:
if is_interactive():
    inp = InputBoxSet(tlabel='multiple dir:')
    display(inp.render())

---
### 3. run the PLIF scoring
---

This returns: 

**For one file:** a dataframe with the scores in  
**For multiple files:** a list of dataframes with the scores in

You can also specify not to write the scores back out to the input sdf files by changing ```write_scores=True``` to ```write_scores=False``` below

In [11]:
if is_interactive():
    ligands_files_dir = inp.inbox.value
    write_scores = True

if os.path.isdir(ligands_files_dir):
    files = glob.glob(os.path.join(ligands_files_dir, '*.sdf'))
        
else:
    files = [ligands_file]
    
plif = PlifScore(protein_pdb=pdb_file, 
                 reference_file=reference_file)

if len(files)==1:
    df = plif.score_conformers(file=ligands_file, write=write_scores, score_col=score_col)
    
else:
    from joblib import Parallel, delayed
    import multiprocessing
    from tqdm import tqdm
    import pandas as pd
    
    num_cores = multiprocessing.cpu_count()
    df_list = Parallel(n_jobs=num_cores)(delayed(plif.score_conformers)(
        file=f, write=write_scores, score_col=score_col
    ) for f in tqdm(files))

  0%|          | 0/499 [00:00<?, ?it/s]

  2%|▏         | 8/499 [00:01<01:18,  6.24it/s]

  2%|▏         | 9/499 [00:08<18:04,  2.21s/it]

  2%|▏         | 10/499 [00:09<14:52,  1.83s/it]

  2%|▏         | 11/499 [00:11<15:21,  1.89s/it]

...

 99%|█████████▉| 495/499 [16:40<00:09,  2.31s/it]

 99%|█████████▉| 496/499 [16:40<00:05,  1.76s/it]

100%|█████████▉| 497/499 [16:41<00:02,  1.30s/it]

100%|█████████▉| 498/499 [16:46<00:02,  2.38s/it]

100%|██████████| 499/499 [16:46<00:00,  1.91s/it]